# Data Wrangling

## 2.1 Introduction 


The purpose of this project is to come up with a recommendation model and suggest the most frequently bought items together.

In this section, we want to handle the data anomalies by removing the NaN and other add or remove other columns from the dataset.

## 2.2 Import Libraries

Placing all the libraries that are required for Data Wrangling are going to be placed in the first first cell. 

In [2]:
import pandas as pd
import numpy as np
import os
import warnings 
warnings.filterwarnings('ignore')

#### Loading the file

- The dataset has two years worth of data consisting of 1067371 rows and is divided across 2 different sheets. 
    - 2009-2010 Sheet
    - 2010-2011 Sheet

In [3]:
retail_data_09_10 = pd.read_excel('../data/raw/online_retail_II.xlsx',sheet_name= 'Year 2009-2010')
print("Shape of 2009-2010 dataset - {}".format(retail_data_09_10.shape))
display(retail_data_09_10.head())



Shape of 2009-2010 dataset - (525461, 8)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [4]:
retail_data_10_11 = pd.read_excel('../data/raw/online_retail_II.xlsx',sheet_name= 'Year 2010-2011')
print("Shape of 2010-2011 dataset - {}".format(retail_data_10_11.shape))
display(retail_data_10_11.head())

Shape of 2010-2011 dataset - (541910, 8)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


* Both datasets have ~520k rows with more rows in 2010-2011 sheet. This could be mainly due to the fact that, there was organic growth in the shop.

#### Combining Data from both datasets

Our step going forward will be to combine data from both the sheets into a single dataframe so further analysis can be performed easily. 

In [5]:
retail_data = pd.concat([retail_data_09_10,retail_data_10_11]).reset_index()

Combined data summary will help us in identifying the outliers, the data types of the columns and how the numeric datatype is distributed across the dataset.

In [6]:
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 9 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   index        1067371 non-null  int64         
 1   Invoice      1067371 non-null  object        
 2   StockCode    1067371 non-null  object        
 3   Description  1062989 non-null  object        
 4   Quantity     1067371 non-null  int64         
 5   InvoiceDate  1067371 non-null  datetime64[ns]
 6   Price        1067371 non-null  float64       
 7   Customer ID  824364 non-null   float64       
 8   Country      1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 73.3+ MB


In [7]:
retail_data.describe()

,index,Quantity,Price,Customer ID
count,1.067371e+06,1.067371e+06,1.067371e+06,824364.000000
mean,2.669056e+05,9.938898e+00,4.649388e+00,15324.638504
std,1.541715e+05,1.727058e+02,1.235531e+02,1697.464450
min,0.000000e+00,-8.099500e+04,-5.359436e+04,12346.000000
25%,1.334210e+05,1.000000e+00,1.250000e+00,13975.000000
50%,2.668420e+05,3.000000e+00,2.100000e+00,15255.000000
75%,4.002635e+05,1.000000e+01,4.150000e+00,16797.000000
max,5.419090e+05,8.099500e+04,3.897000e+04,18287.000000


* From the above analysis we can definitely see that there are a few missing rows in `CustomerId` and `Description`.
* The dataset contains negative Quantity and Price as well. Further analysis will be required inorder to understand what that information is trying to tell us. 

## 2.3 Handle Missing Values

In this section, we are going to see if the missing values can be imputed using a mean or median strategy or we need to drop those rows. 

In [8]:
retail_data.isna().sum()

index               0
Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

In [9]:
missing  = pd.concat([retail_data.isna().sum(),100 * retail_data.isna().mean()],axis = 1)
missing.columns = ['count','%']
missing.sort_values(by='count',ascending=False)

,count,%
Customer ID,243007,22.766873
Description,4382,0.410541
index,0,0.000000
Invoice,0,0.000000
StockCode,0,0.000000
Quantity,0,0.000000
InvoiceDate,0,0.000000
Price,0,0.000000
Country,0,0.000000


`CustomerID` has the most missing values, at around ~23%. Along with that we also see that there are atleast 0.45% of items missing the `Description`. The description is a required column which tells us the item that was sold to the customer. 
`CustomerID` - uniquely identifies a particular customer. 

In [10]:
retail_data.Description.unique()

array(['15CM CHRISTMAS GLASS BALL 20 LIGHTS', 'PINK CHERRY LIGHTS',
       ' WHITE CHERRY LIGHTS', ..., 'mixed up',
       'CREAM HANGING HEART T-LIGHT HOLDER',
       'PAPER CRAFT , LITTLE BIRDIE'], dtype=object)

In [11]:
nan_desc_values = retail_data[retail_data.Description.isna()]
nan_desc_values.head()

,index,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
470,470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.0,NaN,United Kingdom
3114,3114,489655,20683,NaN,-44,2009-12-01 17:26:00,0.0,NaN,United Kingdom
3161,3161,489659,21350,NaN,230,2009-12-01 17:39:00,0.0,NaN,United Kingdom
3731,3731,489781,84292,NaN,17,2009-12-02 11:45:00,0.0,NaN,United Kingdom
4296,4296,489806,18010,NaN,-770,2009-12-02 12:42:00,0.0,NaN,United Kingdom


#### Identifying different combinations for finding a pattern to eliminate NaN values. 

We are trying to find records with -ve Qty and compare that with the NaN values we have in the dataset for `CustomerId` and `Description`.

In [12]:
neg_qty_df = retail_data[(retail_data.Quantity < 0)]

print("# of records with Negative Qty in the dataset - {}".format(neg_qty_df.shape))
neg_qty_df.isna().sum()

# of records with Negative Qty in the dataset - (22950, 9)


index             0
Invoice           0
StockCode         0
Description    2689
Quantity          0
InvoiceDate       0
Price             0
Customer ID    4206
Country           0
dtype: int64

We can see that there are 22,950 rows with negative qty. However only 4206 rows have `null` CustomerID. 

* Therefore, we see that there is no relationship between -ve Qty and `NaN` CustomerId. 


Since, having null CustomerId doesnt help us with the end goal of recommendation. We are going to remove all the `NaN` CustomerId's from the dataset and then try and replace description based on `StockCode` from other rows of the dataset. 

In [13]:
new_df = retail_data[~(retail_data['Customer ID'].isna())]

In [14]:
##Finding the meaning behind '-' in quantity.  C504185
invoice_data = new_df[(new_df['Customer ID'] == 13196.0) & (new_df.StockCode == 84292) ]
invoice_data.head()

,index,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
148231,148231,503515,84292,ROSE PINK METAL FOLDING CHAIR,4,2010-04-01 12:54:00,7.95,13196.0,United Kingdom
156012,156012,C504185,84292,ROSE PINK METAL FOLDING CHAIR,-1,2010-04-12 09:37:00,7.95,13196.0,United Kingdom


In [15]:
print("Shape after dropping NaN rows - {}".format(new_df.shape))
print(new_df.info())

Shape after dropping NaN rows - (824364, 9)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 824364 entries, 0 to 1067370
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   index        824364 non-null  int64         
 1   Invoice      824364 non-null  object        
 2   StockCode    824364 non-null  object        
 3   Description  824364 non-null  object        
 4   Quantity     824364 non-null  int64         
 5   InvoiceDate  824364 non-null  datetime64[ns]
 6   Price        824364 non-null  float64       
 7   Customer ID  824364 non-null  float64       
 8   Country      824364 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 62.9+ MB
None


From the above info method we can see that there are no more missing values in the dataset. 

## 2.4 Explore Data

In [16]:
new_df.Country.value_counts()

United Kingdom          741301
Germany                  17624
EIRE                     16195
France                   14202
Netherlands               5140
Spain                     3811
Belgium                   3123
Switzerland               3064
Portugal                  2504
Australia                 1913
Channel Islands           1664
Italy                     1534
Norway                    1455
Sweden                    1345
Cyprus                    1176
Finland                   1049
Austria                    938
Denmark                    817
Greece                     663
Japan                      582
USA                        535
Poland                     535
Unspecified                524
United Arab Emirates       386
Singapore                  346
Israel                     324
Malta                      299
Iceland                    253
Canada                     228
Lithuania                  189
RSA                        123
Brazil                      94
Thailand

In [20]:
new_df.StockCode.value_counts()

85123A    5322
22423     3777
85099B    3446
84879     2794
20725     2755
          ... 
21766        1
37477C       1
20939        1
72751C       1
90011E       1
Name: StockCode, Length: 4646, dtype: int64

In [99]:
desc_list = new_df.Description.unique()
for unique_elem in desc_list:

    print(unique_elem)

15CM CHRISTMAS GLASS BALL 20 LIGHTS
PINK CHERRY LIGHTS
 WHITE CHERRY LIGHTS
RECORD FRAME 7" SINGLE SIZE 
STRAWBERRY CERAMIC TRINKET BOX
PINK DOUGHNUT TRINKET POT 
SAVE THE PLANET MUG
FANCY FONT HOME SWEET HOME DOORMAT
CAT BOWL 
DOG BOWL , CHASING BALL DESIGN
HEART MEASURING SPOONS LARGE
LUNCHBOX WITH CUTLERY FAIRY CAKES 
DOOR MAT BLACK FLOCK 
LOVE BUILDING BLOCK WORD
HOME BUILDING BLOCK WORD
ASSORTED COLOUR BIRD ORNAMENT
 PEACE WOODEN BLOCK LETTERS
CHRISTMAS CRAFT WHITE FAIRY 
HEART IVORY TRELLIS LARGE
HEART FILIGREE DOVE LARGE
FULL ENGLISH BREAKFAST PLATE
PIZZA PLATE IN BOX
BLACK DINER WALL CLOCK
SET OF 3 BLACK FLYING DUCKS
AREA PATROLLED METAL SIGN
PLEASE ONE PERSON  METAL SIGN
BATH BUILDING BLOCK WORD
CLASSIC WHITE FRAME
SMALL MARSHMALLOWS PINK BOWL
BISCUITS SMALL BOWL LIGHT BLUE
SCOTTIE DOG HOT WATER BOTTLE
CHRISTMAS CRAFT HEART DECORATIONS
CHRISTMAS CRAFT HEART STOCKING 
PARTY CONE CHRISTMAS DECORATION 
PEACE SMALL WOOD LETTERS
JOY LARGE WOOD LETTERS
CINAMMON & ORANGE WREATH
EUCAL

In [100]:
new_df.shape

(824364, 9)

In [101]:
#Records with 0 Price. 
zero_price = new_df[new_df.Price == 0.0]

zero_price.groupby(by=['Price','Description']).size().reset_index(name='counts').sort_values(by='counts',ascending=False)

,Price,Description,counts
34,0.0,Manual,7
19,0.0,CHRISTMAS PUDDING TRINKET POT,2
58,0.0,This is a test product.,2
47,0.0,REGENCY CAKESTAND 3 TIER,2
49,0.0,ROUND CAKE TIN VINTAGE GREEN,2
...,...,...,...
25,0.0,HANGING METAL BIRD BATH,1
26,0.0,HANGING METAL HEART LANTERN,1
27,0.0,HEART GARLAND RUSTIC PADDED,1
28,0.0,IVORY KITCHEN SCALES,1


In [112]:
zero_price.head()

,index,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
4674,4674,489825,22076,6 RIBBONS EMPIRE,12,2009-12-02 13:34:00,0.0,16126.0,United Kingdom
6781,6781,489998,48185,DOOR MAT FAIRY CAKE,2,2009-12-03 11:19:00,0.0,15658.0,United Kingdom
16107,16107,490727,M,Manual,1,2009-12-07 16:38:00,0.0,17231.0,United Kingdom
18738,18738,490961,22065,CHRISTMAS PUDDING TRINKET POT,1,2009-12-08 15:25:00,0.0,14108.0,United Kingdom
18739,18739,490961,22142,CHRISTMAS CRAFT WHITE FAIRY,12,2009-12-08 15:25:00,0.0,14108.0,United Kingdom


In [107]:
new_df = new_df[~(new_df.Price == 0.0)]
new_df.shape

(824293, 9)

#### Splitting datetime column into more meaningful ones. 
* `InvoiceDate` column needs to be split into different sections like Date,Month,Year, Day of the week (eg Sunday,Monday), IsWeekend and IsHoliday. 

In [108]:
new_df['Day_of_week'] = pd.to_datetime(new_df.loc[:,'InvoiceDate']).dt.day_name();
new_df['Date'] = pd.to_datetime(new_df.loc[:,'InvoiceDate']).dt.date;
new_df['Hour'] = pd.to_datetime(new_df.loc[:,'InvoiceDate']).dt.hour;
new_df['Month'] = pd.to_datetime(new_df.loc[:,'InvoiceDate']).dt.month;
new_df['Year'] = pd.to_datetime(new_df.loc[:,'InvoiceDate']).dt.year;
new_df['Day'] = pd.to_datetime(new_df.loc[:,'InvoiceDate']).dt.day;
new_df['Weeknumber'] = pd.to_datetime(new_df.loc[:,'InvoiceDate']).dt.week;

In [109]:
new_df.head()

,index,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Day_of_week,Date,Hour,Month,Year,Day,Weeknumber
0,0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,Tuesday,2009-12-01,7,12,2009,1,49
1,1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,Tuesday,2009-12-01,7,12,2009,1,49
2,2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,Tuesday,2009-12-01,7,12,2009,1,49
3,3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,Tuesday,2009-12-01,7,12,2009,1,49
4,4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,Tuesday,2009-12-01,7,12,2009,1,49


In [110]:
new_df.drop(columns=['InvoiceDate','index'], inplace=True);

new_df.head()

,Invoice,StockCode,Description,Quantity,Price,Customer ID,Country,Day_of_week,Date,Hour,Month,Year,Day,Weeknumber
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95,13085.0,United Kingdom,Tuesday,2009-12-01,7,12,2009,1,49
1,489434,79323P,PINK CHERRY LIGHTS,12,6.75,13085.0,United Kingdom,Tuesday,2009-12-01,7,12,2009,1,49
2,489434,79323W,WHITE CHERRY LIGHTS,12,6.75,13085.0,United Kingdom,Tuesday,2009-12-01,7,12,2009,1,49
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10,13085.0,United Kingdom,Tuesday,2009-12-01,7,12,2009,1,49
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25,13085.0,United Kingdom,Tuesday,2009-12-01,7,12,2009,1,49


#### Save Data

In [111]:
new_df.to_csv('..\data\processed\\OnlineRetail_Cleaned.csv', index=False)

#### Summary from the observed data.

* We see that there are 4646 unique products in the dataset. And `0.41%` of the rows have missing description. 
* In the `Country` column, we see there are 43 unique rows and 756 records are associated with `Unspecified` country. 
* 1820 records with Price set 0.0 
 